<a href="https://colab.research.google.com/github/protasiofernando/health_insurance/blob/main/RAG_Jornada_do_Pesquisador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG com LangChain para a Jornada do Pesquisador

Fluxo do notebook:
1. Instalar e importar as bibliotecas corretas do LangChain.
2. Testar modelos
3. Carregar o texto da Jornada do Pesquisador.
4. Quebrar o texto em chunks e criar uma base vetorial com FAISS.
5. Construir uma chain de RAG
6. Fazer perguntas, recebendo respostas da base de conhecimento da Jornada do Pesquisador com interface Gradio

In [1]:
!pip install -q requests==2.32.4

In [2]:
!pip install -q \
  "langchain>=0.2.16" \
  langchain-community \
  langchain-text-splitters \
  faiss-cpu \
  pypdf \
  sentence-transformers \
  llama-cpp-python \
  ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


## 3. Imports principais do LangChain


# Nova seção

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print('✔ Imports realizados com sucesso.')

✔ Imports realizados com sucesso.


## 4. Criando o LLM do DTI Pesquisa

Vamos criar um `ChatOpenAI` e, mais adiante, usaremos um **prompt de sistema**
para orientá-lo a responder como um fncionário da DTI Pesquisa

In [4]:
!pip install ctransformers

In [5]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/OpenHermes-2.5-Mistral-7B-GGUF",
    model_file="openhermes-2.5-mistral-7b.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,

    # CONTROLE DE TOKENS E EXECUÇÃO
    context_length=4096,
    temperature=0.2,
    top_p=0.9
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

openhermes-2.5-mistral-7b.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

## 5. Carregando o arquivo Jornada do Pesquisador

In [6]:
!pip install -q docx2txt

In [8]:
from langchain_community.document_loaders import Docx2txtLoader

# 1. Define the path to your Word document
# Ensure 'Jornada do Pesquisador.docx' is uploaded to your Colab environment
docx_file_path = "/content/Jornada do Pesquisador.docx"

try:
    # 2. Create an instance of Docx2txtLoader
    loader = Docx2txtLoader(docx_file_path)

    # 3. Call the .load() method to load the document content
    docs = loader.load()

    # 4. Print the number of loaded documents and display the first 500 characters
    print(f"Páginas carregadas: {len(docs)}")
    if docs:
        print("Exemplo de texto da primeira página:\n")
        print(docs[0].page_content[:500])
    else:
        print("Nenhum documento carregado.")
except FileNotFoundError:
    print(f"Erro: O arquivo '{docx_file_path}' não foi encontrado. Por favor, verifique o caminho e se o arquivo foi carregado corretamente.")
except Exception as e:
    print(f"Ocorreu um erro ao carregar o arquivo: {e}")

Páginas carregadas: 1
Exemplo de texto da primeira página:

Jornada dos Pesquisadores

Guia prático para começar, estruturar e rodar sua pesquisa com apoio da TI

Este documento foi elaborado para apoiar quem está iniciando sua trajetória de pesquisa na FGV, reunindo orientações essenciais para o primeiro contato com os recursos tecnológicos disponíveis. É comum que, nesse momento inicial, surjam dúvidas sobre acessos, ferramentas, dados e infraestrutura. Esse guia existe justamente para esclarecer esses pontos.

A equipe de DTI Pesquisa atua como parcei


In [9]:
import re

for doc in docs:
    # 1. Substitui qualquer sequência de caracteres de espaço (incluindo espaços, tabs, novas linhas) por um único espaço
    doc.page_content = re.sub(r'\s+', ' ', doc.page_content)
    # 2. Remove espaços em branco do início e do fim do conteúdo
    doc.page_content = doc.page_content.strip()
print('✔ Tabulações, espaços duplos e outros excessos de espaçamento removidos do texto dos documentos.')

✔ Tabulações, espaços duplos e outros excessos de espaçamento removidos do texto dos documentos.


## 6. Quebrando o texto em *chunks* (recortes menores)

O texto será quebrado em pedaços menores, com um pouco de overlap entre eles para o RAG funcionar bem

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400,
)

splits = text_splitter.split_documents(docs)
print(f"Quantidade de chunks: {len(splits)}")
print("\nExemplo de chunk:\n")
print(splits[0].page_content[:700])

Quantidade de chunks: 11

Exemplo de chunk:

Jornada dos Pesquisadores Guia prático para começar, estruturar e rodar sua pesquisa com apoio da TI Este documento foi elaborado para apoiar quem está iniciando sua trajetória de pesquisa na FGV, reunindo orientações essenciais para o primeiro contato com os recursos tecnológicos disponíveis. É comum que, nesse momento inicial, surjam dúvidas sobre acessos, ferramentas, dados e infraestrutura. Esse guia existe justamente para esclarecer esses pontos. A equipe de DTI Pesquisa atua como parceira ao longo de toda a jornada, com o objetivo de viabilizar e destravar pesquisas por meio da oferta de soluções computacionais adequadas, além da prospecção contínua de tecnologias inovadoras que amplie


In [11]:
if splits:
    print("\nÚltimo chunk:\n")
    print(splits[-1].page_content)
else:
    print("Nenhum chunk disponível.")


Último chunk:

ser acessíveis apenas por um número mínimo de pessoas com autorização específica e protegidos por medidas de segurança excepcionais. Exemplos de dados secretos são: dados de alto valor estratégico, dados críticos para a sobrevivência da organização ou do país, dados que envolvam riscos de vida ou ameaças à integridade física de pessoas. 7. Boas práticas e segurança Aguardando o Quintella para incluir algum guia


## 7. Criando a base vetorial com FAISS

Agora vamos:
1. Gerar embeddings para cada chunk com
2. Guardar esses vetores em uma base FAISS
3. Criar um retriever para buscar os chunks mais relevantes.


In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from sentence_transformers import CrossEncoder
from langchain_core.runnables import RunnableLambda

# 1) Embeddings (Hugging Face / Sentence-Transformers)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True},  # melhora cosine similarity
)

# 2) Vetorização + FAISS
vectorstore = FAISS.from_documents(splits, embeddings)

# 3) Re-ranking multilíngue + Retriever
reranker = CrossEncoder("BAAI/bge-reranker-v2-m3")

TOP_K = 12   # busca inicial no FAISS
RERANK_K = 5 # aqui escolho quantos trechos finais enviar ao LLM

def rerank_retrieve(query: str):
    # Recupera candidatos do FAISS (com score)
    candidates = vectorstore.similarity_search_with_score(query, k=TOP_K)
    docs = [doc for doc, _ in candidates]

    # Re-rank com CrossEncoder: score(query, passage)
    pairs = [(query, d.page_content) for d in docs]
    scores = reranker.predict(pairs)

    # Ordena e devolve os melhores
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked[:RERANK_K]]

# Retriever
retriever = RunnableLambda(rerank_retrieve)

print("✔ Base vetorial criada com FAISS + re-ranking (CrossEncoder) usando Hugging Face.")


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✔ Base vetorial criada com FAISS + re-ranking (CrossEncoder) usando Hugging Face.


### 7.1. Testando a recuperação de trechos

In [15]:
pergunta_teste = "O que é a sala de sigilo?"

# Em versões recentes, o retriever é um Runnable → usamos .invoke
docs_relevantes = retriever.invoke(pergunta_teste)

print(f"Chunks recuperados: {len(docs_relevantes)}\n")
for i, d in enumerate(docs_relevantes, start=1):
    print(f"--- Chunk {i} (página {d.metadata.get('page', 'N/A')}) ---")
    print(d.page_content[:500])
    print()

Chunks recuperados: 5

--- Chunk 1 (página N/A) ---
dados e processamento. 5.3 Uso de nuvem pública A nuvem é indicada quando não é possível atender a demanda com os produtos on-premisse. É importante entender que o uso de plataformas de nuvem pública gera custos e exige cuidados de segurança e governança de dados. As plataformas oferecidas são AWS, Google Cloud (GCP) e Azure. (entender melhor como usar, como pedir, etc....) 5.4 Sala de Sigilo A Sala de Sigilo é um local onde podem ser avaliados dados sigilosos segundo rígidas normas e segurança,

--- Chunk 2 (página N/A) ---
domínio e não compartilhado com entidade externa) Possíveis ambiente para publicar resultados File server (interno) Dataverse (apenas resultado classificado nível 1) Portal web (apenas resultado classificado nível 1) Intranet (pode qualquer dado nivel 1 ou 2) Nível 3 - confidencial Dados que, se divulgados sem autorização, podem causar graves danos à reputação, à competitividade, à segurança ou à continuidade da o

## 8. Construindo a *chain* de RAG com um assistente da DTI Pesquisa

Vamos montar um *pipeline* (chain) com a API de **Runnables**:

1. Entrada: pergunta da usuária (`question`).
2. `retriever` → busca os trechos mais relevantes da cartilha.
3. `format_docs` → junta os trechos em um único *contexto*.
4. `ChatPromptTemplate` → define o papel do modelo como ginecologista sênior.
5. `Chat Mistral` → gera a resposta em português, com empatia.
6. `StrOutputParser` → converte a resposta em string simples.


In [20]:
from langchain_core.runnables import RunnableLambda

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# Função para converter ChatPromptValue (lista de mensagens) em uma string simples para o ctransformers LLM
def combine_messages_into_string_prompt(chat_prompt_value):
    combined_prompt = ""
    for message in chat_prompt_value.messages:
        combined_prompt += message.content + "\n\n"
    return combined_prompt.strip()

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Você é um expert em TI que responde apenas com base no contexto fornecido.
Se a resposta não estiver no contexto, diga que não encontrou a informação. Você não deve fazer perguntas ao usuário.

Regras de comunicação:
- Fale SEMPRE em português do Brasil, com linguagem clara e objetiva
- Baseie suas respostas APENAS nas informações do CONTEXTO fornecido.
- Se algo não estiver claro no contexto, diga que não há dados suficientes para responder
e recomende uma leitura completa do Guia Jornada do Pesquisador
"""),
    ("user", """
CONTEXTO (trechos da Jornada do Pesquisador):
{context}

PERGUNTA DO PESQUISADOR:
{question}
"""),
])

parser = StrOutputParser()

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | RunnableLambda(combine_messages_into_string_prompt) # Nova etapa para formatar a entrada para o LLM do ctransformers
    | llm
    | parser
)

print('✔ Chain de RAG criada.')

✔ Chain de RAG criada.


## 9. Testando perguntas

In [21]:
# Exemplo de pergunta

pergunta_teste = "O que é a sala de sigilo?"

print("PERGUNTA DO PESQUISADOR:")
print(pergunta_teste)
print("\nRESPOSTA DO ASSISTENTE (RAG):\n")
resposta = rag_chain.invoke(pergunta_teste)
print(resposta)
print("\n")

PERGUNTA DO PESQUISADOR:
O que é a sala de sigilo?

RESPOSTA DO ASSISTENTE (RAG):

 Qual sua função na pesquisa?




In [22]:
print('Instalando Gradio...')
!pip install -q gradio
print('✔ Gradio instalado.')

Instalando Gradio...
✔ Gradio instalado.


In [23]:
import gradio as gr
import base64

def rag_interface(question):
    """
    Função que será usada pelo Gradio para invocar a chain de RAG.
    """
    if not question.strip():
        return "Por favor, digite sua pergunta."

    if "rag_chain" not in globals():
        return "Erro: A 'rag_chain' não foi encontrada. Por favor, execute as células anteriores do notebook para defini-la."

    resposta = rag_chain.invoke(question)
    return resposta


# Caminho local da imagem
local_assistant_image_path = "/content/Robô Assistente.png"

# Codificar a imagem para Base64
try:
    with open(local_assistant_image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
    assistant_image_data_uri = f"data:image/png;base64,{encoded_image}"
except FileNotFoundError:
    print(f"Aviso: O arquivo de imagem '{local_assistant_image_path}' não foi encontrado. Usando um placeholder.")
    assistant_image_data_uri = "https://cdn-icons-png.flaticon.com/512/9440/9440624.png"


# CSS customizado
custom_css = """
.gradio-container { background-color: #f8f8f8; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; }

.header-wrap {
  display: flex;
  justify-content: center;
  align-items: center;
  gap: 10px;
  margin-bottom: 12px;
}
.header-title { color: #2c3e50; font-size: 2.0em; font-weight: 700; margin: 0; }
.gradio-description { color: #7f8c8d; font-size: 1.05em; text-align: center; margin-bottom: 22px; }

.section-title-wrap {
  text-align: center;
  margin-bottom: 10px;
}
.section-title-wrap h3 { margin: 6px 0 0 0; color: #34495e; }

.equal-height-row { align-items: stretch; }
.equal-height-col { display: flex; flex-direction: column; height: 100%; }

.box-wrap { display: flex; flex-direction: column; flex: 1; }
.box-wrap .gradio-textbox { flex: 1; }

.textbox-eq textarea {
  min-height: 330px !important;  /* altura igual para pergunta e resposta */
}

.gradio-button { background-color: #3498db; color: white; border-radius: 5px; padding: 10px 20px; font-size: 1.05em; }
.gradio-button:hover { background-color: #2980b9; }
"""

# Interface
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    # Cabeçalho com imagem importada
    gr.HTML(
        f"""
        <div class="header-wrap">
          <img src="{assistant_image_data_uri}" width="50" style="display:block;">
          <h1 class="header-title">Assistente DTI Pesquisa</h1>
        </div>
        <div class="gradio-description">
          Faça perguntas sobre a Jornada do Pesquisador e receba respostas baseadas no documento.
        </div>
        """
    )

    with gr.Row(elem_classes=["equal-height-row"]):
        with gr.Column(scale=1, elem_classes=["equal-height-col"]):
            gr.HTML('<div class="section-title-wrap"><h3>Sua Pergunta</h3></div>')

            with gr.Group(elem_classes=["box-wrap"]):
                question_input = gr.Textbox(
                    placeholder="Digite sua dúvida aqui",
                    label="",
                    show_label=False,
                    elem_classes=["textbox-eq"]
                )

            submit_button = gr.Button("Obter Resposta")

        with gr.Column(scale=2, elem_classes=["equal-height-col"]):
            # Removida a imagem aqui, mantendo apenas o título
            gr.HTML(
                """
                <div class="section-title-wrap">
                  <h3>Resposta do Assistente</h3>
                </div>
                """
            )

            with gr.Group(elem_classes=["box-wrap"]):
                output_text = gr.Textbox(
                    label="",
                    show_label=False,
                    interactive=False,
                    elem_classes=["textbox-eq"]
                )

    submit_button.click(fn=rag_interface, inputs=question_input, outputs=output_text)

    gr.Markdown("<br><hr style='border-top: 1px solid #ccc;'><br>")
    gr.Markdown(
        "<footer><p style='text-align: center; color: #7f8c8d;'>"
        "Esta ferramenta é para fins informativos. Para informações oficiais, consulte o documento completo da Jornada do Pesquisador."
        "</p></footer>"
    )

print("Iniciando a interface Gradio")
demo.launch(share=True)

/tmp/ipython-input-3528705506.py:66: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3528705506.py:66: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:


Iniciando a interface Gradio com estilo aprimorado...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e01b48399ebbbd155e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
